In [ ]:
# ============================================================
# BLOQUE 1 — Imports y carga de entorno
# ============================================================

import os                      # Para leer variables de entorno (OPENAI_API_KEY, etc.)
import json                    # Para parsear argumentos JSON de tool_calls
import sqlite3                 # Para usar SQLite (BD local en un archivo)
from dotenv import load_dotenv # Para cargar variables desde un archivo .env
from openai import OpenAI      # Cliente OpenAI (también sirve con base_url para Ollama OpenAI-compatible)
import gradio as gr            # Para montar una UI tipo chat en local


In [ ]:
# ============================================================
# BLOQUE 2 — Inicialización: API key, modelo y cliente
# ============================================================

load_dotenv(override=True)  # Carga variables desde .env y sobreescribe si ya existen

openai_api_key = os.getenv("OPENAI_API_KEY")  # Lee la API key del entorno
if openai_api_key:                             # Si existe
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")  # Muestra solo el inicio
else:                                          # Si no existe
    print("OpenAI API Key not set")            # Aviso

MODEL = "gpt-4.1-mini"  # Modelo que vas a usar (en remoto con OpenAI)
openai = OpenAI()       # Crea el cliente usando la key del entorno

# --- Alternativa: usar Ollama local en modo OpenAI-compatible ---
# MODEL = "llama3.1:8b"
# openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")


In [ ]:
# ============================================================
# BLOQUE 3 — Mensaje de sistema (reglas del asistente)
# ============================================================

system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""


In [ ]:
# ============================================================
# BLOQUE 4 — Base de datos SQLite: archivo y tabla
# ============================================================

DB = "prices.db"  # Nombre del archivo SQLite (se crea si no existe)

with sqlite3.connect(DB) as conn:                                  # Abre conexión al archivo DB
    cursor = conn.cursor()                                         # Crea cursor para ejecutar SQL
    cursor.execute(                                                # Crea la tabla si no existe
        "CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)"
    )
    conn.commit()                                                  # Confirma la creación (persistencia)


In [ ]:
# ============================================================
# BLOQUE 5 — Tools reales (funciones Python que hacen trabajo útil)
# ============================================================

def get_ticket_price(city: str) -> str:
    """
    Tool: consulta el precio de una ciudad en la tabla 'prices' (SQLite).
    Devuelve una frase (string) para que el LLM la use.
    """
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)  # Log inmediato para depurar

    with sqlite3.connect(DB) as conn:                                    # Abre conexión a SQLite
        cursor = conn.cursor()                                           # Cursor para consulta
        cursor.execute(                                                  # Consulta parametrizada (evita inyección SQL)
            "SELECT price FROM prices WHERE city = ?",
            (city.lower(),)                                              # Tuple de 1 elemento (coma final)
        )
        result = cursor.fetchone()                                       # Devuelve (precio,) o None

    # Si hay resultado, result[0] es el valor de la columna price
    if result:
        return f"Ticket price to {city} is ${result[0]}"
    # Si no hay fila, devolvemos mensaje de ausencia
    return "No price data available for this city"


def set_ticket_price(city: str, price: float) -> str:
    """
    Tool: inserta o actualiza el precio de una ciudad en SQLite.
    Devuelve un string de confirmación.
    """
    print(f"DATABASE TOOL CALLED: Setting price for {city} -> {price}", flush=True)  # Log inmediato

    with sqlite3.connect(DB) as conn:                                                # Abre conexión a SQLite
        cursor = conn.cursor()                                                       # Cursor SQL
        cursor.execute(                                                              # UPSERT (insert o update)
            "INSERT INTO prices (city, price) VALUES (?, ?) "
            "ON CONFLICT(city) DO UPDATE SET price = ?",
            (city.lower(), price, price)                                             # 3 parámetros (2 insert + 1 update)
        )
        conn.commit()                                                                # Confirma cambios en DB

    return f"Updated ticket price for {city} to ${price}"                            # Texto para el LLM


In [ ]:
# ============================================================
# BLOQUE 6 — (Opcional) Precargar datos en la DB
# ============================================================

seed_prices = {  # Diccionario de ejemplo: ciudad -> precio numérico
    "london": 799,
    "paris": 899,
    "tokyo": 1420,
    "sydney": 2999,
    "madrid":1500,
    "Hondarribia":1970
}

for city, price in seed_prices.items():  # Recorre todas las entradas
    set_ticket_price(city, price)        # Inserta/actualiza en DB usando la tool


In [ ]:
# ============================================================
# BLOQUE 7 — Esquemas (schemas) de tools: descripción para el modelo
# ============================================================
# Importante: los nombres de parámetros del schema deben coincidir
# con los nombres de argumentos de las funciones Python, porque luego
# llamaremos a func(**args).

get_price_schema = {  # Schema para get_ticket_price(city)
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {  # Debe llamarse "city" porque la función Python espera city
                "type": "string",
                "description": "Destination city (e.g. London, Paris)"
            }
        },
        "required": ["city"],              # Obligatorio
        "additionalProperties": False      # No permite parámetros extra
    }
}

set_price_schema = {  # Schema para set_ticket_price(city, price)
    "name": "set_ticket_price",
    "description": "Set or update the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {  # Debe llamarse "city" porque la función Python espera city
                "type": "string",
                "description": "Destination city (e.g. London, Paris)"
            },
            "price": {  # Debe llamarse "price" porque la función Python espera price
                "type": "number",
                "description": "Ticket price (numeric), e.g. 899"
            }
        },
        "required": ["city", "price"],     # Ambos obligatorios
        "additionalProperties": False      # No permite parámetros extra
    }
}

tools = [  # Lista de tools que pasas al modelo
    {"type": "function", "function": get_price_schema},  # Tool 1
    {"type": "function", "function": set_price_schema},  # Tool 2
]


In [ ]:
# ============================================================
# BLOQUE 8 — Registro pythónico (lista blanca): nombre_tool -> función
# ============================================================
# Esto elimina la necesidad de if/elif por cada tool.
# Solo ejecutamos herramientas que estén explícitamente registradas.

TOOL_REGISTRY = {
    "get_ticket_price": get_ticket_price,  # Autoriza la tool "get_ticket_price"
    "set_ticket_price": set_ticket_price,  # Autoriza la tool "set_ticket_price"
}


In [ ]:
# ============================================================
# BLOQUE 9 — Handler genérico de tool calls (sin if por tool)
# ============================================================

def handle_tool_calls(assistant_message):
    """
    Recibe el mensaje del modelo que contiene tool_calls.
    Ejecuta cada tool call usando TOOL_REGISTRY y devuelve una lista
    de mensajes role="tool" que se añaden al historial.
    """
    tool_responses = []  # Aquí acumulamos las respuestas tipo "tool"

    for tool_call in assistant_message.tool_calls:             # Recorre cada tool solicitada
        tool_name = tool_call.function.name                    # Nombre de tool solicitada (string)
        func = TOOL_REGISTRY.get(tool_name)                    # Busca la función en la lista blanca

        # Parseo de argumentos: tool_call.function.arguments es un string JSON
        raw_args = tool_call.function.arguments or "{}"        # Si viene None/vacío, usamos {}
        args = json.loads(raw_args)                            # Convierte JSON -> dict

        if func is None:                                       # Si la tool no está autorizada/registrada
            output = f"Tool '{tool_name}' is not available."   # Mensaje seguro (no ejecuta nada)
        else:
            output = func(**args)                              # Llamada pythónica: kwargs desde args dict

        # Construimos el mensaje "tool" que el modelo entenderá en la siguiente llamada
        tool_responses.append({
            "role": "tool",                                    # Rol especial "tool"
            "content": output,                                 # Texto resultante de la herramienta
            "tool_call_id": tool_call.id                       # Vincula respuesta con llamada concreta
        })

    return tool_responses                                      # Devuelve lista de respuestas tool


In [ ]:
# ============================================================
# BLOQUE 10 — Función chat para Gradio con tool calling encadenado
# ============================================================

def chat(message, history):
    """
    Esta función la llama Gradio en cada turno.
    - message: texto del usuario
    - history: historial de mensajes (user/assistant) proporcionado por Gradio
    """
    # Normaliza history a una lista de dicts con solo role y content
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    # Construye el prompt completo: system + history + nuevo user message
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # Llamada inicial al modelo con tools disponibles
    response = openai.chat.completions.create(
        model=MODEL,            # Modelo seleccionado
        messages=messages,      # Historial completo
        tools=tools             # Descripción de herramientas que puede solicitar
    )

    # Bucle: mientras el modelo pida tools, las ejecutamos y volvemos a llamar
    while response.choices[0].finish_reason == "tool_calls":
        assistant_msg = response.choices[0].message            # Mensaje del modelo con tool_calls
        tool_msgs = handle_tool_calls(assistant_msg)           # Ejecuta tool calls -> lista role="tool"

        messages.append(assistant_msg)                         # Añade el mensaje del modelo (petición tool)
        messages.extend(tool_msgs)                             # Añade las respuestas de tools

        # Nueva llamada al modelo, ahora con el resultado de las tools en el historial
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

    # Cuando ya no pide tools, devolvemos el texto final del assistant
    return response.choices[0].message.content


In [ ]:
# ============================================================
# BLOQUE 11 — Lanzar la interfaz en Gradio
# ============================================================

gr.ChatInterface(fn=chat, type="messages").launch()
